# Data Warehouse

In [158]:
# Import libraries
import pandas as pd
import datetime
import psycopg2

## Get data from datalake01

In [159]:
# Connect to database
ENDPOINT = 'datalake01.cbahnwmpgx2t.us-east-1.rds.amazonaws.com'
DB_NAME = 'datalake01'
USERNAME = 'postgrebbex'
PASSWORD = 'databasebab7!'

try:
    print("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))

except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)

host=datalake01.cbahnwmpgx2t.us-east-1.rds.amazonaws.com dbname=datalake01 user=postgrebbex password=databasebab7!


### Queries

In [160]:
sql_weather = """
select id as weather_id, description, main, temperature, temp_fees_like as temp_feels_like , date_trunc('hour', time_request  + interval '30 minute') nearest_hour
from public.weather w 
left join (
	select weather_id, description, main 
	from public.weather_info wi 
	group by 1,2,3
) info_weather on info_weather.weather_id = w.weather_id 
where date_trunc('hour', time_request  + interval '30 minute')::date = current_date -1
"""

sql_providers = """
select provider_id, "name" as provider_name, vehicle_type 
from public.providers p 
where vehicle_type in ('E-Scooter', 'Bike', 'E-Bike', 'E-CargoBike')
group by 1,2,3
"""

sql_stations = """ 
select station_id, "name" as station_name, lat, lon, s.provider_id, provider_name, vehicle_type
from stations s 
left join (select provider_id, "name" as provider_name, vehicle_type 
from public.providers p 
group by 1,2,3) p on s.provider_id = p.provider_id 
where vehicle_type in ('E-Scooter', 'Bike', 'E-Bike', 'E-CargoBike')
"""

### Data loading

In [161]:
weather = pd.read_sql(sql_weather, conn)
providers = pd.read_sql(sql_providers, conn)
stations = pd.read_sql(sql_stations, conn)

C:\Users\misch\AppData\Local\Temp\ipykernel_23060\1801735763.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  weather = pd.read_sql(sql_weather, conn)
C:\Users\misch\AppData\Local\Temp\ipykernel_23060\1801735763.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  providers = pd.read_sql(sql_providers, conn)
C:\Users\misch\AppData\Local\Temp\ipykernel_23060\1801735763.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stations = pd.read_sql(sql_stations, conn)


In [162]:
# delete duplicates
stations = stations[~stations.duplicated()]

In [163]:
# close db-connection
cur.close()
conn.close()

## Get data from cycleonmg

In [201]:
# Connect to database
ENDPOINT = "cycleonmg.cd41ai1dxxcn.us-east-1.rds.amazonaws.com"
DB_NAME = "cycleonmg"
USERNAME = "mischi"
PASSWORD = "123456789"

try:
    print("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))

except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)

host=cycleonmg.cd41ai1dxxcn.us-east-1.rds.amazonaws.com dbname=cycleonmg user=mischi password=123456789


### Queries

To decide: 
* include location or not?
* What vehicles to count for free float (exclude disabled?)

In [200]:
sql_station_based = """ 
select date_trunc('hour', last_updated + interval '30 minute') nearest_hour, station_id as location_id, sum(number_vehicles_available) as no_of_vehicles
from public.station_status ss 
where date_trunc('hour', last_updated + interval '30 minute')::date = current_date -1
group by 1,2
"""

sql_free_float = """ 
select provider_id, date_trunc('hour', last_updated + interval '30 minute') nearest_hour, count(*) as no_of_vehicles
from free_floating_systems ffs 
where date_trunc('hour', last_updated + interval '30 minute')::date = current_date -1
and is_disabled = false
group by 1,2
"""

sql_free_float_loc = """ 
select vehicle_id as location_id, provider_id, is_disabled, is_reserved, date_trunc('hour', last_updated + interval '30 minute') nearest_hour, count(*) as no_of_vehicles
from public.free_floating_systems ffs 
where date_trunc('hour', last_updated + interval '30 minute')::date = current_date -1
group by 1,2,3,4,5
"""

sql_location_ff = """
select vehicle_id as location_id, lat, lon, date_trunc('hour', last_updated + interval '30 minute') nearest_hour
from public.free_floating_systems
where date_trunc('hour', last_updated + interval '30 minute')::date = current_date -1
"""

### Data Loading

In [202]:
station_based = pd.read_sql(sql_station_based, conn)
free_float = pd.read_sql(sql_free_float, conn)
free_float_loc = pd.read_sql(sql_free_float_loc, conn)
location_free_float = pd.read_sql(sql_location_ff, conn)

C:\Users\misch\AppData\Local\Temp\ipykernel_23060\501714529.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  station_based = pd.read_sql(sql_station_based, conn)
C:\Users\misch\AppData\Local\Temp\ipykernel_23060\501714529.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  free_float = pd.read_sql(sql_free_float, conn)
C:\Users\misch\AppData\Local\Temp\ipykernel_23060\501714529.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  free_float_loc = pd.read_sql(sql_free_float_loc, conn)
C:\Users\misch\AppData\Local\Temp\ipyk

In [196]:
# close db-connection
cur.close()
conn.close()

## Joining Data Sources

In [203]:
join_free_float = free_float.merge(providers, on='provider_id', how = 'left').merge(weather, on = 'nearest_hour')
join_free_float_loc = free_float_loc.merge(providers, on='provider_id', how = 'left').merge(weather, on = 'nearest_hour')
join_stations_based_loc = station_based.merge(stations, left_on = 'location_id', right_on = 'station_id', how = 'left').merge(weather, on = 'nearest_hour')

In [204]:
# prepare number of vehicles by timestamp and provider
station_based_grouped = station_based.merge(stations, left_on = 'location_id', right_on = 'station_id', how = 'left').groupby(['nearest_hour', 'provider_id'], as_index= False).sum()[['nearest_hour', 'provider_id', 'no_of_vehicles']]
join_stations_based = station_based_grouped.merge(providers, on='provider_id', how = 'left').merge(weather, on = 'nearest_hour')
join_stations_based

,nearest_hour,provider_id,no_of_vehicles,provider_name,vehicle_type,weather_id,description,main,temperature,temp_feels_like
0,2023-11-15 00:00:00,carvelo2go,14,Carvelo2go,E-CargoBike,147,broken clouds,Clouds,10.22,9.41
1,2023-11-15 00:00:00,publibike,643,Publibike,Bike,147,broken clouds,Clouds,10.22,9.41
2,2023-11-15 00:00:00,publiebike,623,PubliBike,E-Bike,147,broken clouds,Clouds,10.22,9.41
3,2023-11-15 01:00:00,carvelo2go,14,Carvelo2go,E-CargoBike,148,moderate rain,Rain,9.51,6.02
4,2023-11-15 01:00:00,publibike,644,Publibike,Bike,148,moderate rain,Rain,9.51,6.02
...,...,...,...,...,...,...,...,...,...,...
67,2023-11-15 22:00:00,publibike,645,Publibike,Bike,169,broken clouds,Clouds,7.27,7.27
68,2023-11-15 22:00:00,publiebike,581,PubliBike,E-Bike,169,broken clouds,Clouds,7.27,7.27
69,2023-11-15 23:00:00,carvelo2go,17,Carvelo2go,E-CargoBike,170,few clouds,Clouds,6.47,5.02
70,2023-11-15 23:00:00,publibike,647,Publibike,Bike,170,few clouds,Clouds,6.47,5.02


### Fact Table

In [205]:
column_names_loc = ['location_id', 'provider_id', 'nearest_hour', 'weather_id', 'no_of_vehicles']
column_names = ['provider_id', 'nearest_hour', 'weather_id', 'no_of_vehicles']

fact_free_float = join_free_float[column_names]
fact_station_based  = join_stations_based[column_names]

fact_free_float_loc = join_free_float_loc[column_names_loc]
fact_station_based_loc  = join_stations_based_loc[column_names_loc]

fact = pd.concat([fact_free_float, fact_station_based])
fact_loc = pd.concat([fact_free_float_loc, fact_station_based_loc])
fact

,provider_id,nearest_hour,weather_id,no_of_vehicles
0,bird-zurich-escooter,2023-11-15 00:00:00,147,807
1,tier,2023-11-15 00:00:00,147,726
2,voiscooters.com,2023-11-15 00:00:00,147,861
3,bird-zurich-escooter,2023-11-15 01:00:00,148,809
4,tier,2023-11-15 01:00:00,148,731
...,...,...,...,...
67,publibike,2023-11-15 22:00:00,169,645
68,publiebike,2023-11-15 22:00:00,169,581
69,carvelo2go,2023-11-15 23:00:00,170,17
70,publibike,2023-11-15 23:00:00,170,647


### Dimension Tables

#### Location

In [206]:
column_names = ['location_id', 'lat', 'lon', 'nearest_hour']

location_station_based  = join_stations_based_loc[column_names]

location = pd.concat([location_free_float, location_station_based])
location

,location_id,lat,lon,nearest_hour
0,bird-zurich-escooter:fa275655-0256-46a5-8d17-6...,47.379520,8.537279,2023-11-15 00:00:00
1,tier:153374,47.369324,8.491393,2023-11-15 21:00:00
2,tier:153405,47.391440,8.514714,2023-11-15 21:00:00
3,tier:153633,47.353970,8.511642,2023-11-15 21:00:00
4,tier:154027,47.412600,8.540414,2023-11-15 21:00:00
...,...,...,...,...
9259,publiebike:95,47.377148,8.523818,2023-11-15 23:00:00
9260,publiebike:96,47.378548,8.531836,2023-11-15 23:00:00
9261,publiebike:97,47.377155,8.510488,2023-11-15 23:00:00
9262,publiebike:98,47.373110,8.533314,2023-11-15 23:00:00


#### Weather

In [207]:
column_names = ['weather_id', 'description', 'main', 'temperature', 'temp_feels_like']

weather_dim = weather[column_names]
weather_dim

,weather_id,description,main,temperature,temp_feels_like
0,169,broken clouds,Clouds,7.27,7.27
1,168,broken clouds,Clouds,7.4,5.03
2,167,broken clouds,Clouds,7.51,6.24
3,166,broken clouds,Clouds,7.88,5.62
4,164,broken clouds,Clouds,8.69,5.65
5,162,broken clouds,Clouds,9.97,8.16
6,161,broken clouds,Clouds,10.54,9.71
7,157,broken clouds,Clouds,9.13,6.92
8,155,broken clouds,Clouds,8.77,6.27
9,154,broken clouds,Clouds,8.64,6.54


#### Provider

Doesn't need to be updated regularly. This data can be loaded in bulk once the collection is over (or once in the beginning and then just updated)

In [208]:
column_names = ['provider_id', 'provider_name', 'vehicle_type']

provider_dim = providers[column_names]

provider_dim

,provider_id,provider_name,vehicle_type
0,bird-platform-partner-jmfleets-bulle,JM Fleets,E-Scooter
1,donkey_kreuzlingen,Regivelo.ch,Bike
2,bird-zurich-escooter,Bird,E-Scooter
3,donkey_sion,Donkey Republic,Bike
4,donkey_le_locle,Donkey Republic,Bike
5,velospot,Velospot,Bike
6,nextbike_ch,Nextbike,Bike
7,donkey_yverdon-les-bains,Donkey Republic,Bike
8,donkey_ge,Donkey Republic,Bike
9,publiebike,PubliBike,E-Bike


#### Time

In [157]:
time = weather[['nearest_hour']]
time['weekday'] = time['nearest_hour'].apply(lambda x: x.weekday())

time

C:\Users\misch\AppData\Local\Temp\ipykernel_23060\2002376973.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time['weekday'] = time['nearest_hour'].apply(lambda x: x.weekday())


,nearest_hour,weekday
0,2023-11-11 14:00:00,5
1,2023-11-11 10:00:00,5
2,2023-11-11 09:00:00,5
3,2023-11-11 05:00:00,5
4,2023-11-11 23:00:00,5
5,2023-11-11 04:00:00,5
6,2023-11-11 03:00:00,5
7,2023-11-11 12:00:00,5
8,2023-11-11 11:00:00,5
9,2023-11-11 22:00:00,5


## Load Data to Data Warehouse